# Fund Lightining channel with PBST

In [ ]:
%load_ext autoreload
%autoreload 2

## Init Bitcoin RPC client

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
rpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
rpc.getblockcount()

## Mine coins

In [ ]:
import bitcoin.core as bc

In [ ]:
rpc.createwallet('miner')
# rpc.loadwallet('miner')

In [ ]:
rpc.getbalance()

In [ ]:
minerAddr = rpc.getnewaddress("coinbase")
minerAddr

In [ ]:
# coinbase transactions are locked for 100 blocks
# need to mine more than x to make sure lnd is synced with the chain
_ = list(rpc.generatetoaddress(400, minerAddr))

In [ ]:
rpc.getbalance()

In [ ]:
rpc.unloadwallet('miner')

## Generate addresses for Ali and Bob

### Ali

In [ ]:
rpc.createwallet('ali')

In [ ]:
aliFundAddr = rpc.getnewaddress('ali fund')
aliFundAddr

In [ ]:
aliPremiumAddr = rpc.getnewaddress('ali premium')
aliPremiumAddr

In [ ]:
rpc.unloadwallet('ali')

### Bob

In [ ]:
rpc.createwallet('bob')

In [ ]:
bobFundAddr = rpc.getnewaddress("bob fund")
bobFundAddr

In [ ]:
rpc.unloadwallet('bob')

## Fund Ali and Bob

In [ ]:
rpc.loadwallet('miner')

In [ ]:
fundTxId = rpc.sendmany("",
                        {aliFundAddr: 10*bc.COIN, bobFundAddr: 11*bc.COIN},
                        comment="Funding Ali and Bob")

In [ ]:
# confirm transactions
_ = list(rpc.generatetoaddress(6, minerAddr))

In [ ]:
rpc.unloadwallet('miner')

### Check funds

In [ ]:
rpc.loadwallet('ali')
aliUnspent = rpc.listunspent(addrs=[aliFundAddr])
rpc.unloadwallet('ali')
aliUnspent[0]['amount'] / bc.COIN

In [ ]:
rpc.loadwallet('bob')
bobUnspent = rpc.listunspent(addrs=[bobFundAddr])
rpc.unloadwallet('bob')
bobUnspent[0]['amount'] / bc.COIN

## Connect Ali and Bob

```
# on Bob's node
lncli --network=regtest getinfo
# 02a2c3ea47fb6fa83dacdadff57cd769c22ecc7c785b9b28e2f36834f82172946e
```

In [ ]:
bobLNPubKey = '02a2c3ea47fb6fa83dacdadff57cd769c22ecc7c785b9b28e2f36834f82172946e'

```
# on Ali's node
lncli --network=regtest connect 02a2c3ea47fb6fa83dacdadff57cd769c22ecc7c785b9b28e2f36834f82172946e@bob-lnd

lncli --network=regtest listpeers
```

## Create funding PSBT

In [ ]:
import bitcoin.core.script as bs
import bitcoin.wallet as bw

### Start opening channel with pbst

In [ ]:
fundAmount = int(0.15 * bc.COIN) # by default should be less than 0.16777215

In [ ]:
# on Ali's node
f"lncli --network=regtest openchannel --node_key='{bobLNPubKey}' --local_amt={int(fundAmount)} --psbt"

In [ ]:
# taken from the output of the last command
lnChanAddr = bw.P2WSHBitcoinAddress('bcrt1qhr9rz7pvdaa6rhak266e8lvgkn46xxm6v0ya64s6083cnrcz480qms8vdg')
lnChanAddr

### Get Inputs

In [ ]:
aliTxin = bc.CTxIn(aliUnspent[0]['outpoint'])
aliTxin

In [ ]:
bobTxin = bc.CTxIn(bobUnspent[0]['outpoint'])
bobTxin

In [ ]:
aliUnspent[0]['amount']

In [ ]:
bobUnspent[0]['amount']

In [ ]:
txFee = int(0.01 * bc.COIN)
aliPremium = int(1.0 * bc.COIN)
changeAmount = aliUnspent[0]['amount'] + bobUnspent[0]['amount'] - fundAmount - txFee - aliPremium
f'fundAmount = {fundAmount / bc.COIN}  changeAmount = {changeAmount / bc.COIN}', 

### Create outputs

In [ ]:
premiumTxout = bc.CTxOut(aliPremium, aliPremiumAddr.to_scriptPubKey())
premiumTxout

In [ ]:
# for simplicity send full change to ali's fund address
# in reality we would have 2 change addresses for each party
changeTxout = bc.CTxOut(changeAmount, aliFundAddr.to_scriptPubKey())
changeTxout

In [ ]:
lnChanTxOut = bc.CTxOut(fundAmount, lnChanAddr.to_scriptPubKey())
lnChanTxOut

### Create transaction

In [ ]:
fundTx = bc.CMutableTransaction([aliTxin, bobTxin], [premiumTxout, changeTxout, lnChanTxOut])
fundTx

### Convert to PBST

In [ ]:
from bitcoin.rpc import hexlify, unhexlify

In [ ]:
pbst = rpc._proxy._call('converttopsbt', hexlify(fundTx.serialize()))
pbst

### Ali signs funding tx

In [ ]:
rpc.loadwallet('ali')

In [ ]:
res = rpc._proxy._call('walletprocesspsbt', pbst)
aliPsbt = res['psbt']
res

In [ ]:
rpc.unloadwallet('ali')

### Bob signs funding tx

In [ ]:
rpc.loadwallet('bob')

In [ ]:
res = rpc._proxy._call('walletprocesspsbt', aliPsbt)
bobPsbt = res['psbt']
res

In [ ]:
rpc.unloadwallet('bob')

In [ ]:
# paste psbt (base64) back into ali's lnd session

### Finalize PBST

In [ ]:
res = rpc._proxy._call('finalizepsbt', bobPsbt)
res

In [ ]:
# paste psbt (hex) back into ali's lnd session

In [ ]:
# confirm transactions
_ = list(rpc.generatetoaddress(6, minerAddr))

### Check LN channel

on ali's or bob's nodes
```
lncli --network=regtest listchannels
```

send payment

```
# on bob's node
lncli --network=regtest addinvoice --amt=10000
```

```
# on ali's node
lncli --network=regtest sendpayment --pay_req=lnbcrt100u1ps8ugyapp5xcxrv2469eskszs9eueksxv0jrdcgrvfrl6v50g5k23gj39af9asdqqcqzpgsp5f4sa0ma7lkdgu04zmaqr45889eu6cn0jl72p55usx772lk6tu5vq9qyyssqfnlvntn0wp2qzf8rejjugjtcxy79zu94mqxu433ck0m0axd5l9x3vsrf03et2engau3lgmeuj37vkvl8xvv5rgqxhkscy2lcxkk0pqcps3nlgd
```

```
on bob's node
lncli --network=regtest channelbalance
```